In [1]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

from src.webscraping2 import (
    scrape_to_dataframe,
    convert_columns,
    combine_home_visitor,  
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)

import json

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')

**Load API keys**

In [2]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

try:
    SCRAPINGANT_API_KEY = os.environ['SCRAPINGANT_API_KEY']
except:
    raise Exception('Set environment variable SCRAPINGANT_API_KEY')

**Scrape New Completed Games and Format Them**

In [3]:

def get_new_games()-> pd.DataFrame:

    # set search for yesterday's games
    DAYS = 1
    SEASON = "" #no season will cause website to default to current season, format is "2022-23"
    TODAY = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
    LASTWEEK = (TODAY - timedelta(days=DAYS))
    DATETO = TODAY.strftime("%m/%d/%y")
    DATEFROM = LASTWEEK.strftime("%m/%d/%y")


    df = scrape_to_dataframe(api_key = SCRAPINGANT_API_KEY, Season=SEASON, DateFrom=DATEFROM, DateTo=DATETO)

    df = convert_columns(df)

    print(df.info())
    df = combine_home_visitor(df)

    return df

df_new = get_new_games()

# get the SEASON of the last game in the database
# this will used when constructing rows for prediction
SEASON = df_new['SEASON'].max()

df_new




https://www.nba.com/stats/teams/boxscores?DateFrom=01/27/23&DateTo=01/28/23
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   HOME            10 non-null     int64         
 1   GAME_DATE_EST   10 non-null     datetime64[ns]
 2   HOME_TEAM_WINS  10 non-null     int64         
 3   PTS             10 non-null     int64         
 4   FG_PCT          10 non-null     float64       
 5   FG3_PCT         10 non-null     float64       
 6   FT_PCT          10 non-null     float64       
 7   REB             10 non-null     int64         
 8   AST             10 non-null     int64         
 9   TEAM_ID         10 non-null     object        
 10  GAME_ID         10 non-null     object        
dtypes: datetime64[ns](1), float64(3), int64(5), object(2)
memory usage: 960.0+ bytes
None


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2023-01-27,1,141,53.1,41.9,61.3,50,32,1610612749,22200736,131,55.1,39.4,71.4,38,22,1610612754,2022
1,2023-01-27,0,100,40.9,24.2,84.2,47,22,1610612763,22200737,111,51.2,37.1,66.7,45,31,1610612750,2022
2,2023-01-27,0,100,44.3,27.6,77.8,37,27,1610612739,22200739,112,52.6,55.6,88.0,46,19,1610612760,2022
3,2023-01-27,0,105,47.8,35.3,84.4,40,21,1610612753,22200738,110,45.9,30.3,84.6,37,28,1610612748,2022
4,2023-01-27,0,117,45.5,36.4,61.9,46,24,1610612761,22200740,129,55.6,41.9,73.3,43,40,1610612744,2022


**Retrieve todays games**

In [4]:
#retrieve list of teams playing today

# get today's games on NBA schedule
matchups, game_ids = get_todays_matchups(api_key = SCRAPINGANT_API_KEY)


print(matchups)
print(game_ids)


[['1610612748', '1610612766'], ['1610612754', '1610612763'], ['1610612746', '1610612739'], ['1610612740', '1610612749']]
['22200752', '22200753', '22200754', '22200755']


**Create Rows for Today's Games with Empty Stats**

In [5]:
# append today's matchups to the new games dataframe


df_today = df_new.drop(df_new.index) #empty copy of df_new with same columns
for i, matchup in enumerate(matchups):
    game_details = {'HOME_TEAM_ID': matchup[1], 
                    'VISITOR_TEAM_ID': matchup[0], 
                    'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                    'GAME_ID': int(game_ids[i]),                       
                    'SEASON': SEASON,
                    } 
    game_details_df = pd.DataFrame(game_details, index=[i])
    # append to new games dataframe
    df_today = pd.concat([df_today, game_details_df], ignore_index = True)

#blank rows will be filled with 0 to prevent issues with feature engineering
df_today = df_today.fillna(0) 

df_today


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2023-01-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612766,22200752,0.0,0.0,0.0,0.0,0.0,0.0,1610612748,2022
1,2023-01-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612763,22200753,0.0,0.0,0.0,0.0,0.0,0.0,1610612754,2022
2,2023-01-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612739,22200754,0.0,0.0,0.0,0.0,0.0,0.0,1610612746,2022
3,2023-01-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612749,22200755,0.0,0.0,0.0,0.0,0.0,0.0,1610612740,2022


**Access Feature Store**

In [6]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3350


Connected. Call `.close()` to terminate connection gracefully.


**Access Feature Group**

In [7]:
rolling_stats_fg = fs.get_feature_group(
    name="rolling_stats",
    version=1,
)

**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [8]:
BASE_FEATURES = ['game_date_est',
 'game_id',
 'home_team_id',
 'visitor_team_id',
 'season',
 'pts_home',
 'fg_pct_home',
 'ft_pct_home',
 'fg3_pct_home',
 'ast_home',
 'reb_home',
 'pts_away',
 'fg_pct_away',
 'ft_pct_away',
 'fg3_pct_away',
 'ast_away',
 'reb_away',
 'home_team_wins',
]

ds_query = rolling_stats_fg.select(BASE_FEATURES)
df_old = ds_query.read()
df_old


2023-01-28 12:03:38,040 INFO: USE `nba_predictor_featurestore`
2023-01-28 12:03:38,466 INFO: SELECT `fg0`.`game_date_est` `game_date_est`, `fg0`.`game_id` `game_id`, `fg0`.`home_team_id` `home_team_id`, `fg0`.`visitor_team_id` `visitor_team_id`, `fg0`.`season` `season`, `fg0`.`pts_home` `pts_home`, `fg0`.`fg_pct_home` `fg_pct_home`, `fg0`.`ft_pct_home` `ft_pct_home`, `fg0`.`fg3_pct_home` `fg3_pct_home`, `fg0`.`ast_home` `ast_home`, `fg0`.`reb_home` `reb_home`, `fg0`.`pts_away` `pts_away`, `fg0`.`fg_pct_away` `fg_pct_away`, `fg0`.`ft_pct_away` `ft_pct_away`, `fg0`.`fg3_pct_away` `fg3_pct_away`, `fg0`.`ast_away` `ast_away`, `fg0`.`reb_away` `reb_away`, `fg0`.`home_team_wins` `home_team_wins`
FROM `nba_predictor_featurestore`.`rolling_stats_1` `fg0`


,game_date_est,game_id,home_team_id,visitor_team_id,season,pts_home,fg_pct_home,ft_pct_home,fg3_pct_home,ast_home,reb_home,pts_away,fg_pct_away,ft_pct_away,fg3_pct_away,ast_away,reb_away,home_team_wins
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2009-03-15,20800989,1610612755,1610612748,2008,85,0.423096,0.666992,0.293945,24,40,77,0.412109,0.695801,0.262939,22,38,1
4,2004-11-17,20400109,1610612751,1610612760,2004,68,0.413086,0.889160,0.000000,9,37,79,0.406006,0.758789,0.333008,12,33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23536,2023-01-27,22200739,1610612760,1610612739,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0
23537,2023-01-26,22200732,1610612741,1610612766,2022,96,40.687500,81.812500,16.000000,16,43,111,46.406250,83.875000,21.906250,24,52,0
23538,2023-01-26,22200735,1610612759,1610612746,2022,100,45.093750,64.312500,33.312500,24,42,138,58.406250,76.187500,47.406250,26,37,0
23539,2023-01-26,22200733,1610612739,1610612745,2022,113,45.500000,81.812500,41.687500,29,36,95,51.312500,60.000000,23.796875,24,46,1


**Convert Feature Names back to original mixed case**

In [9]:
df_old = convert_feature_names(df_old)
df_old

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2009-03-15,20800989,1610612755,1610612748,2008,85,0.423096,0.666992,0.293945,24,40,77,0.412109,0.695801,0.262939,22,38,1
4,2004-11-17,20400109,1610612751,1610612760,2004,68,0.413086,0.889160,0.000000,9,37,79,0.406006,0.758789,0.333008,12,33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23536,2023-01-27,22200739,1610612760,1610612739,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0
23537,2023-01-26,22200732,1610612741,1610612766,2022,96,40.687500,81.812500,16.000000,16,43,111,46.406250,83.875000,21.906250,24,52,0
23538,2023-01-26,22200735,1610612759,1610612746,2022,100,45.093750,64.312500,33.312500,24,42,138,58.406250,76.187500,47.406250,26,37,0
23539,2023-01-26,22200733,1610612739,1610612745,2022,113,45.500000,81.812500,41.687500,29,36,95,51.312500,60.000000,23.796875,24,46,1


**Update Yesterday's Matchup Predictions with New Final Results**

In [10]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results

#df_old = df_old.set_index('GAME_ID')
#df_old.update(df_new.set_index('GAME_ID'))
#df_old = df_old.reset_index()  

# another approach is to simply drop the rows that were used for prediction yesterday
# which are games that have 0 points for home team
# and then append the new rows to the dataframe
df_old = df_old[df_old['PTS_home'] != 0]
df_old = pd.concat([df_old, df_new], ignore_index = True)

df_old

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2009-03-15,20800989,1610612755,1610612748,2008,85,0.423096,0.666992,0.293945,24,40,77,0.412109,0.695801,0.262939,22,38,1
4,2004-11-17,20400109,1610612751,1610612760,2004,68,0.413086,0.889160,0.000000,9,37,79,0.406006,0.758789,0.333008,12,33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23484,2023-01-27,22200736,1610612749,1610612754,2022,141,53.100000,61.300000,41.900000,32,50,131,55.100000,71.400000,39.400000,22,38,1
23485,2023-01-27,22200737,1610612763,1610612750,2022,100,40.900000,84.200000,24.200000,22,47,111,51.200000,66.700000,37.100000,31,45,0
23486,2023-01-27,22200739,1610612739,1610612760,2022,100,44.300000,77.800000,27.600000,27,37,112,52.600000,88.000000,55.600000,19,46,0
23487,2023-01-27,22200738,1610612753,1610612748,2022,105,47.800000,84.400000,35.300000,21,40,110,45.900000,84.600000,30.300000,28,37,0


**Add Today's Matchups for Feature Engineering**

In [11]:
df_combined = pd.concat([df_old, df_today], ignore_index = True)
df_combined

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2017-12-08 00:00:00,21700374,1610612759,1610612738,2017,105.0,0.468994,0.875000,0.295898,16.0,46.0,102.0,0.458008,0.881836,0.289062,14.0,39.0,1.0
1,2013-03-01 00:00:00,21200874,1610612756,1610612737,2012,92.0,0.444092,0.833008,0.455078,16.0,38.0,87.0,0.425049,0.772949,0.347900,21.0,43.0,1.0
2,2005-11-30 00:00:00,20500210,1610612738,1610612755,2005,110.0,0.447998,0.784180,0.250000,24.0,59.0,103.0,0.408936,0.770996,0.308105,21.0,40.0,1.0
3,2009-03-15 00:00:00,20800989,1610612755,1610612748,2008,85.0,0.423096,0.666992,0.293945,24.0,40.0,77.0,0.412109,0.695801,0.262939,22.0,38.0,1.0
4,2004-11-17 00:00:00,20400109,1610612751,1610612760,2004,68.0,0.413086,0.889160,0.000000,9.0,37.0,79.0,0.406006,0.758789,0.333008,12.0,33.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23488,2023-01-27 00:00:00,22200740,1610612761,1610612744,2022,117.0,45.500000,61.900000,36.400000,24.0,46.0,129.0,55.600000,73.300000,41.900000,40.0,43.0,0.0
23489,2023-01-28,22200752,1610612766,1610612748,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
23490,2023-01-28,22200753,1610612763,1610612754,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
23491,2023-01-28,22200754,1610612739,1610612746,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0


**Data Processing**

In [12]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,PLAYOFF,TARGET
0,2017-12-08 00:00:00,21700374,1610612759,1610612738,2017,105.0,0.468994,0.875000,0.295898,16.0,46.0,102.0,0.458008,0.881836,0.289062,14.0,39.0,1.0,0,1.0
1,2013-03-01 00:00:00,21200874,1610612756,1610612737,2012,92.0,0.444092,0.833008,0.455078,16.0,38.0,87.0,0.425049,0.772949,0.347900,21.0,43.0,1.0,0,1.0
2,2005-11-30 00:00:00,20500210,1610612738,1610612755,2005,110.0,0.447998,0.784180,0.250000,24.0,59.0,103.0,0.408936,0.770996,0.308105,21.0,40.0,1.0,0,1.0
3,2009-03-15 00:00:00,20800989,1610612755,1610612748,2008,85.0,0.423096,0.666992,0.293945,24.0,40.0,77.0,0.412109,0.695801,0.262939,22.0,38.0,1.0,0,1.0
4,2004-11-17 00:00:00,20400109,1610612751,1610612760,2004,68.0,0.413086,0.889160,0.000000,9.0,37.0,79.0,0.406006,0.758789,0.333008,12.0,33.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23488,2023-01-27 00:00:00,22200740,1610612761,1610612744,2022,117.0,45.500000,61.900000,36.400000,24.0,46.0,129.0,55.600000,73.300000,41.900000,40.0,43.0,0.0,0,0.0
23489,2023-01-28,22200752,1610612766,1610612748,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0
23490,2023-01-28,22200753,1610612763,1610612754,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0
23491,2023-01-28,22200754,1610612739,1610612746,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0


**Feature Engineering**

In [13]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)
df_combined


#fix type conversion issues with hopsworks
df_combined['TARGET'] = df_combined['TARGET'].astype('int16')
df_combined['HOME_TEAM_WINS'] = df_combined['HOME_TEAM_WINS'].astype('int16')


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

**Insert New Data into Feature Group**

In [14]:

def test():
    # retrieve only new games from the combined dataframe now that feature engineering is complete

    # set index to GAME_ID
    df_combined = df_combined.set_index('GAME_ID')
    df_new = df_new.set_index('GAME_ID')
    
    # retrieve only new games
    df_new = df_combined.loc[df_new.index]

    # reset GAME_ID index back to column
    df_new = df_new.reset_index()

    # convert certain features back to int32 for Hopsworks compatibility
    df_new['GAME_ID'] = df_new['GAME_ID'].astype('int32')
    df_new['HOME_TEAM_WINS'] = df_new['HOME_TEAM_WINS'].astype('int32')
    df_new['TARGET'] = df_new['TARGET'].astype('int32')

    # save new games to Hopsworks feature group
    rolling_stats_fg.insert(df_new, write_options={"wait_for_job" : False})

    df_new 

rolling_stats_fg.insert(df_combined, write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/23493 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3350/jobs/named/rolling_stats_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x2712419aca0>, None)

In [15]:
df_old.tail(20)

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
23469,2023-01-25,22200725,1610612737,1610612760,2022,137,59.50000,78.8125,37.90625,26,46,132,52.00000,68.8125,44.093750,31,37,1
23470,2023-01-25,22200728,1610612761,1610612758,2022,113,49.50000,71.3750,30.00000,28,41,95,47.90625,92.3125,40.593750,29,34,1
23471,2023-01-25,22200724,1610612750,1610612740,2022,111,48.81250,71.3750,37.90625,23,36,102,43.50000,76.1875,31.593750,25,42,1
23472,2023-01-25,22200729,1610612759,1610612747,2022,104,37.68750,88.8750,23.50000,22,48,113,44.40625,76.6875,30.296875,25,56,0
23473,2023-01-25,22200726,1610612763,1610612744,2022,120,47.68750,70.3125,32.31250,26,36,122,50.59375,78.6250,43.812500,33,44,0
23474,2023-01-25,22200723,1610612743,1610612749,2022,99,39.18750,75.0000,32.40625,21,51,107,47.40625,71.3750,43.312500,16,49,0
23475,2023-01-25,22200720,1610612754,1610612753,2022,120,48.18750,79.3125,42.90625,25,32,126,56.09375,76.0000,53.593750,34,34,0
23476,2023-01-25,22200727,1610612762,1610612757,2022,124,52.18750,66.6875,48.81250,32,35,134,57.31250,83.3125,47.187500,26,41,0
23477,2023-01-25,22200722,1610612764,1610612745,2022,108,39.81250,78.3125,31.59375,27,48,103,43.31250,70.0000,17.906250,29,57,1
23478,2023-01-26,22200734,1610612742,1610612756,2022,99,42.90625,71.3750,38.18750,20,43,95,40.68750,75.0000,40.593750,24,50,1


In [16]:
df_combined[df_combined['GAME_ID'] > 22200640]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
23407,2023-01-14,22200647,1610612742,1610612757,2022,119,52.40625,57.90625,47.812500,28,...,3.201563,0.757292,1.000000,-2.571429,-1.8,-2.600000,1.666667,-4.428571,-3.1,-3.200000
23408,2023-01-14,22200646,1610612755,1610612762,2022,118,44.90625,80.81250,43.593750,23,...,4.151563,2.839583,8.333333,2.714286,1.4,2.733333,-7.000000,-2.857143,-5.2,-1.533333
23409,2023-01-14,22200645,1610612739,1610612750,2022,102,48.68750,87.00000,28.593750,24,...,-1.815625,-1.963542,2.000000,-0.142857,0.9,-0.066667,1.000000,4.000000,2.5,1.666667
23410,2023-01-14,22200644,1610612737,1610612761,2022,114,51.18750,94.37500,40.687500,19,...,-0.345312,2.051042,-5.000000,-1.142857,0.0,1.466667,4.000000,4.142857,3.5,2.666667
23411,2023-01-14,22200641,1610612749,1610612748,2022,95,40.50000,66.68750,39.593750,26,...,1.879687,0.347917,4.333333,1.857143,1.0,0.333333,0.333333,6.714286,10.2,9.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23488,2023-01-27,22200737,1610612763,1610612750,2022,100,40.90625,84.18750,24.203125,22,...,0.246875,-1.220833,6.666667,4.000000,3.0,3.400000,4.000000,11.857143,9.9,11.000000
23489,2023-01-28,22200755,1610612749,1610612740,2022,0,0.00000,0.00000,0.000000,0,...,9.910937,3.012500,6.333333,2.142857,2.6,2.400000,7.666667,3.285714,3.4,6.933333
23490,2023-01-28,22200754,1610612739,1610612746,2022,0,0.00000,0.00000,0.000000,0,...,-5.643750,-5.675000,4.666667,3.571429,2.9,2.000000,-5.333333,-3.142857,-4.2,-1.600000
23491,2023-01-28,22200752,1610612766,1610612748,2022,0,0.00000,0.00000,0.000000,0,...,1.392187,-0.266667,-0.333333,2.428571,4.8,4.066667,1.000000,-3.857143,-1.4,1.333333
